# Unified Bedrock Model Manager Demonstration

This notebook demonstrates the **UnifiedModelManager** - a comprehensive solution that integrates regular Amazon Bedrock model data with CRIS (Cross-Region Inference Service) data to provide a single source of truth for model access information.

## Key Features
- 🎯 **Single Source of Truth**: Unified view of model availability across all regions
- 🔄 **Automatic Integration**: Correlates regular model data with CRIS data
- 🌍 **Access Method Detection**: Identifies direct access, CRIS-only, or both options
- 📊 **Comprehensive Querying**: Rich filtering and analysis capabilities
- ⚡ **Smart Recommendations**: Optimal access method suggestions with rationale


## Setup and Imports

In [ ]:
import sys
from pathlib import Path
import logging
import json
from datetime import datetime

# Add the src directory to path for imports
sys.path.append(str(Path.cwd().parent / "src"))

# Import our unified model manager and related classes
from bedrock.UnifiedModelManager import UnifiedModelManager, UnifiedModelManagerError
from bedrock.models.access_method import ModelAccessMethod, ModelAccessInfo
from bedrock.models.unified_structures import UnifiedModelInfo

# Configure logging for better visibility
logging.basicConfig(
    level=logging.INFO,
    format='%(asctime)s - %(name)s - %(levelname)s - %(message)s'
)

print("✅ Imports successful!")
print(f"📁 Working directory: {Path.cwd()}")

## Initialize the Unified Model Manager

The `UnifiedModelManager` serves as the main interface for accessing unified model information.

In [ ]:
# Initialize the UnifiedModelManager
print("🚀 Initializing UnifiedModelManager...")

manager = UnifiedModelManager(
    force_download=False,  # Set to True to always download fresh data
    download_timeout=30
)

print(f"✅ UnifiedModelManager initialized:")
print(f"   📄 JSON output path: {manager.json_output_path}")
print(f"   🔄 Force download: {manager.force_download}")
print(f"\n{manager}")

## Refresh Unified Data

This process will:
1. Download regular Bedrock model documentation
2. Download CRIS model documentation  
3. Correlate and merge the two data sources
4. Create a unified catalog with comprehensive access information

**Note**: This may take a moment as it downloads and processes data from AWS documentation.

In [ ]:
print("📥 Refreshing unified model data...")
print("   This will download and correlate data from multiple sources...")

try:
    # Refresh the unified data
    catalog = manager.refresh_unified_data()
    
    print(f"\n✅ Successfully created unified catalog!")
    print(f"   📊 Total models: {catalog.model_count}")
    print(f"   🕐 Retrieved at: {catalog.retrieval_timestamp}")
    
    # Get correlation statistics
    stats = manager.get_correlation_stats()
    print(f"\n📈 Correlation Statistics:")
    for key, value in stats.items():
        print(f"   {key.replace('_', ' ').title()}: {value}")
    
except UnifiedModelManagerError as e:
    print(f"❌ Error refreshing data: {e}")
    # You can continue with cached data if available
    catalog = manager.load_cached_data()
    if catalog:
        print(f"📂 Using cached data: {catalog.model_count} models")
    else:
        raise

## Basic Catalog Information

Let's explore the basic information about our unified catalog.

In [ ]:
# Basic catalog information
print("📋 Unified Catalog Overview")
print("=" * 40)

# Model names
model_names = manager.get_model_names()
print(f"\n🤖 Available Models ({len(model_names)}):")
for i, name in enumerate(model_names[:15]):  # Show first 15
    print(f"   {i+1:2d}. {name}")
if len(model_names) > 15:
    print(f"   ... and {len(model_names) - 15} more")

# Supported regions
regions = manager.get_all_supported_regions()
print(f"\n🌍 Supported Regions ({len(regions)}):")
for i, region in enumerate(regions[:20]):  # Show first 20
    print(f"   {region}")
if len(regions) > 20:
    print(f"   ... and {len(regions) - 20} more")

# Provider breakdown
print(f"\n🏢 Provider Analysis:")
providers = ["Amazon", "Anthropic", "Meta", "Mistral", "DeepSeek", "Writer"]
for provider in providers:
    provider_models = manager.get_models_by_provider(provider=provider)
    if provider_models:
        print(f"   {provider:12s}: {len(provider_models):2d} models")

## Core Functionality: Model Access Information

This is the heart of the unified system - determining how to access models in different regions.

In [ ]:
def display_access_info(model_name: str, region: str):
    """Helper function to display comprehensive access information for a model-region pair."""
    print(f"\n🔍 Access Information: '{model_name}' in '{region}'")
    print("-" * 60)
    
    # Check if model exists
    if not manager.has_model(model_name):
        print(f"❌ Model '{model_name}' not found in catalog")
        return
    
    # Check availability in region
    available = manager.is_model_available_in_region(model_name=model_name, region=region)
    print(f"   Available in {region}: {'✅ Yes' if available else '❌ No'}")
    
    if not available:
        return
    
    # Get detailed access information
    access_info = manager.get_model_access_info(model_name=model_name, region=region)
    if access_info:
        print(f"   Access Method: {access_info.access_method.value.upper()}")
        
        if access_info.model_id:
            print(f"   🎯 Direct Model ID: {access_info.model_id}")
        
        if access_info.inference_profile_id:
            print(f"   🔄 CRIS Profile ID: {access_info.inference_profile_id}")
        
        # Get recommendation
        recommendation = manager.get_recommended_access(model_name=model_name, region=region)
        if recommendation:
            print(f"   ⭐ Recommended: {recommendation.recommended_access.access_method.value.upper()}")
            print(f"   💡 Rationale: {recommendation.rationale}")
            
            if recommendation.alternatives:
                print(f"   🔀 Alternatives: {len(recommendation.alternatives)} available")
                for alt in recommendation.alternatives:
                    print(f"      - {alt.access_method.value.upper()}")

# Test cases demonstrating different access patterns
test_cases = [
    ("Claude 3 Haiku", "us-east-1"),      # Should have both direct and CRIS
    ("Nova Lite", "us-east-1"),          # Amazon model with multiple access methods
    ("Nova Pro", "eu-west-1"),           # Cross-region availability
    ("DeepSeek-R1", "us-east-1"),        # Newer CRIS-only model
    ("Claude 3 Haiku", "ap-southeast-1"), # CRIS access in APAC
]

print("🎯 Model Access Information Examples")
print("=" * 50)

for model_name, region in test_cases:
    display_access_info(model_name, region)

## Summary and Next Steps

This notebook demonstrates the comprehensive capabilities of the UnifiedModelManager system.

In [ ]:
print("📋 UnifiedModelManager Summary")
print("=" * 35)

print("\n✅ Key Capabilities Demonstrated:")
print("   🎯 Single source of truth for model access information")
print("   🔄 Automatic integration of regular and CRIS model data")
print("   🌍 Comprehensive region and access method analysis")
print("   📊 Rich querying and filtering capabilities")
print("   ⚡ Smart access method recommendations")

print("\n🚀 Next Steps:")
print("   1. Integrate with your application's model selection logic")
print("   2. Use get_model_access_info() to determine optimal access methods")
print("   3. Leverage regional analysis for deployment strategies")
print("   4. Set up automated data refresh for production use")

print("\n📚 Additional Resources:")
print("   - UnifiedModelManager API documentation")
print("   - Model correlation and mapping details")
print("   - CRIS integration best practices")

print("\n🎉 Unified model management system ready for production use!")